1：corpora和dictionary的基本概念和用法
corpora是gensim的一个基本概念，是文档集的表现形式。
举例：
hurry up
rise up
这两篇文档里面共有3个词，如果将这3个词映射到数字，比如说hurry, rise, up 分别对应1,2,3， 那么上述的文档集的一种表现形式可以是：
1,0,1
0,1,1
这种方法有一些弊端，比如只考虑了词频，且不考虑词语间的位置关系。因为第一个文档的两个词分别编号1,3且都只出现了一次，所以第1个和第3个为1，第2个为0；但是当文章单词量极多的时候，这种表达方式就不合适了，需要采用稀疏矩阵的形式

那么，如果将字符串形式的文档转换为上述形式呢？这里就要提到词典的概念，词典是所有文章中所有单词的集合，而且记录了各词的出现次数等信息。


将文档分词分割成词语之后，使用dictionary = corpora.Dictionary(texts)生成词典，并可以使用save函数将词典持久化，生成词典以后corpus=[dictionary.doc2bow(text) for text in texts]转化为向量形式。

In [2]:
from gensim import corpora
from collections import defaultdict
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

C:\Users\Johnson\Anaconda3\envs\py36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#去停用词
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

# 去掉只出现一次的单词
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

In [4]:
dictionary = corpora.Dictionary(texts) #生成地点

In [6]:
#将文档存入字典，字典有很多功能，比如：
# diction.token2id 存放的是单词-id key-value对
# diction.dfs 存放的是单词的出现频率

# dictionary.save('/tmp/deerwester.dict')
corpus = [dictionary.doc2bow(text) for text in texts]
# corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # store to disk, for later use

# 注意最后的corpora.MmCorpus.serialize 将corpus持久化到磁盘中。相反，可以用
# corpus = corpora.MmCorpus('/tmp/deerwester.mm')
# 来读取磁盘中的corpus
# 除了MmCorpus以外，还有其他的格式，例如SvmLightCorpus, BleiCorpus, LowCorpus等等，用法类似。



1.2 dictionary的其他一些用法
dictionary还有其他的一些用法，现罗列一部分
dictionary.filter_n_most_frequent(N) 
过滤掉出现频率最高的N个单词

dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000) 
1.去掉出现次数低于no_below的 
2.去掉出现次数高于no_above的。注意这个小数指的是百分数 
3.在1和2的基础上，保留出现频率前keep_n的单词

dictionary.filter_tokens(bad_ids=None, good_ids=None) 
有两种用法，一种是去掉bad_id对应的词，另一种是保留good_id对应的词而去掉其他词。注意这里bad_ids和good_ids都是列表形式

dictionary.compacity() 
在执行完前面的过滤操作以后，可能会造成单词的序号之间有空隙，这时就可以使用该函数来对词典来进行重新排序，去掉这些空隙。

2:models
在models中，可以对corpus进行进一步的处理，比如使用tf-idf模型，lda模型等，非常强大
在按照之前的方法生成了corpus和dictionary以后，就可以生成模型了
    tfidf_model = models.TfidfModel(corpus)
    
注意，目前只是生成了一个模型，但这是类似与生成器，并不是将对应corpus转换后的结果，对tf-idf模型而言，里面存储有各个单词的词频，文频等信息。想要将文档转换成if-idf模型表示的向量，还要使用一下命令
corpus_tfidf = tfidf_model[corpus]

In [15]:
from gensim import models,similarities

In [12]:
tfidf_model = models.TfidfModel(corpus)
corpus_tfidf = tfidf_model[corpus]

In [13]:
# 对于lda和lsi模型，用法有所不同

lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi_model[corpus_tfidf]

可以看到，这里除了corpus以外，还多了num_topic的选项。这是指的潜在主题(topic)的数目，也等于转成lsi模型以后每个文档对应的向量长度。转化以后的向量在各项的值，即为该文档在该潜在主题的权重。因此lsi和lda的结果也可以看做该文档的文档向量，用于后续的分类，聚类等算法。值得注意的是，id2word是所有模型都有的选项，可以指定使用的词典。

In [ ]:
import os
from gensim import corpora, models, similarities
from pprint import pprint
from matplotlib import pyplot as plt
import logging

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def PrintDictionary(dictionary):
    token2id = dictionary.token2id
    dfs = dictionary.dfs
    token_info = {}
    for word in token2id:
        token_info[word] = dict(
            word = word,
            id = token2id[word],
            freq = dfs[token2id[word]]
        )
    token_items = token_info.values()
    token_items = sorted(token_items, key = lambda x:x['id'])
    print('The info of dictionary: ')
    pprint(token_items)
    print('--------------------------')

def Show2dCorpora(corpus):
    nodes = list(corpus)
    ax0 = [x[0][1] for x in nodes] # 绘制各个doc代表的点
    ax1 = [x[1][1] for x in nodes]
    # print(ax0)
    # print(ax1)
    plt.plot(ax0,ax1,'o')
    plt.show()

if (os.path.exists("/tmp/deerwester.dict")):
    dictionary = corpora.Dictionary.load('/tmp/deerwester.dict')
    corpus = corpora.MmCorpus('/tmp/deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")

PrintDictionary(dictionary)

# 尝试将corpus(bow形式) 转化成tf-idf形式
tfidf_model = models.TfidfModel(corpus) # step 1 -- initialize a model 将文档由按照词频表示 转变为按照tf-idf格式表示
doc_bow = [(0, 1), (1, 1),[4,3]]
doc_tfidf = tfidf_model[doc_bow]

# 将整个corpus转为tf-idf格式
corpus_tfidf = tfidf_model[corpus]
# pprint(list(corpus_tfidf))
# pprint(list(corpus))

## LSI模型 **************************************************
# 转化为lsi模型, 可用作聚类或分类
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lsi = lsi_model[corpus_tfidf]
nodes = list(corpus_lsi)
# pprint(nodes)
lsi_model.print_topics(2) # 打印各topic的含义

# ax0 = [x[0][1] for x in nodes] # 绘制各个doc代表的点
# ax1 = [x[1][1] for x in nodes]
# print(ax0)
# print(ax1)
# plt.plot(ax0,ax1,'o')
# plt.show()

lsi_model.save('/tmp/model.lsi') # same for tfidf, lda, ...
lsi_model = models.LsiModel.load('/tmp/model.lsi')
#  *********************************************************

## LDA模型 **************************************************
lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=2)
corpus_lda = lda_model[corpus_tfidf]
Show2dCorpora(corpus_lsi)
# nodes = list(corpus_lda)
# pprint(list(corpus_lda))

# 此外，还有Random Projections, Hierarchical Dirichlet Process等模型

In [16]:
corpus_simi_matrix = similarities.MatrixSimilarity(corpus_lsi)
# 计算一个新的文本与既有文本的相关度
test_text = "Human computer interaction".split()
test_bow = dictionary.doc2bow(test_text)
test_tfidf = tfidf_model[test_bow]
test_lsi = lsi_model[test_tfidf]
test_simi = corpus_simi_matrix[test_lsi]
print(list(enumerate(test_simi)))

[(0, 0.9991645), (1, 0.9963216), (2, 0.9990505), (3, 0.99886364), (4, 0.99996823), (5, -0.058117405), (6, -0.02158928), (7, 0.0135240555), (8, 0.25163394)]


C:\Users\Johnson\Anaconda3\envs\py36\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
